# 03: Fairness Analysis

This notebook evaluates baseline models for group-level disparities:
- Demographic parity analysis
- Equalized odds assessment
- Precision parity evaluation
- Disaggregated performance metrics by protected attributes
- Visualizations of fairness metrics


In [ ]:
# Install dependencies (for Google Colab)
!pip install fairlearn seaborn -q
